# Predicting Food Truck Locations Using Snowpark ML and XGBoost

In this project, we use the Snowpark ML library and XGBoost to build a classification model that can predict the location of the Freezing Point food truck on any given day. After training the model, we assess its performance using relevant metrics to gauge accuracy and effectiveness. Then we register the model to the Snowflake Model Registry, logging it with its metadata and key metrics for easy access and management within Snowflake.

----


### Preliminaries: Installing Libraries

Install the required libraries

In [1]:
!pip install snowflake-ml-python
!pip install snowflake-snowpark-python

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


### Importing Libraries

Importing the required libraries

In [2]:
import csv
from credential import params

# Import the required libraries
from snowflake.snowpark import Session
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.functions import col
from snowflake.ml.modeling import preprocessing
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.registry import Registry

To create a Snowflake session, a dictionary of credentials containing account name, user name, password, and other details can be provided. To simplify this process and avoid exposing credentials, a separate [credential.py](credential.py) file was created. This file is then imported as a library to directly pull the dictionary of credentials.

*IMPORTANT: In a Production environment, it's crucial to secure passwords using secret managers and/or OAuth.*

### Creating the Dataset

Here’s the neighborhood visiting pattern the truck follows:

In January, the truck goes to neighborhood 1 on the 1st, 8th, 15th, 22nd, and 29th, and neighborhood 2 the other days.

From February through November, it goes to:

- Neighborhood 1 on the 1st
- Neighborhood 2 on the 2nd
- Neighborhood 3 on the 3rd
- Neighborhood 4 on the 4th
- Neighborhood 5 on the 5th
- Neighborhood 6 on the 6th
- Neighborhood 7 on the 7th

Then loops back again after visiting neighborhood 7:

- Neighborhood 1 on the 8th
- Neighborhood 2 on the 9th, and so on, until the next month starts when it then restarts the pattern.

Every December, it only goes to neighborhood 8.

In [3]:
month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [4]:
pre = {}

for i,month_length in enumerate(month_days):
    month = i + 1

    for day in range(1,month_length+1):
        
        # In January, it goes to neighborhood 1 on Mondays, and neighborhood 2 the other days.
        if ((month) == 1):
            if (day) % 7 == 1:
                pre[(month,day)] = 1
            else:
                pre[(month,day)] = 2
                
        # From February through November, it goes to neighborhood 1 on the 1st, 2 on the 2nd, 3 on the 3rd,
        # 4 on the 4th, 5 on the 5th, 6 on the 6th, and 7 on the 7th, 1 on the 8th, 2 on the 9th, etc.
        elif ((month) <= 11):
            pre[(month,day)] = ((day-1) % 7) + 1

        # Every December, it only goes to neighborhood 8.
        elif ((month) == 12):
            pre[(month,day)] = 8

By utilizing this neighborhood visiting pattern, we generate data for an entire year.

In [5]:
# See what the pre dictionary looks like
pre

{(1, 1): 1,
 (1, 2): 2,
 (1, 3): 2,
 (1, 4): 2,
 (1, 5): 2,
 (1, 6): 2,
 (1, 7): 2,
 (1, 8): 1,
 (1, 9): 2,
 (1, 10): 2,
 (1, 11): 2,
 (1, 12): 2,
 (1, 13): 2,
 (1, 14): 2,
 (1, 15): 1,
 (1, 16): 2,
 (1, 17): 2,
 (1, 18): 2,
 (1, 19): 2,
 (1, 20): 2,
 (1, 21): 2,
 (1, 22): 1,
 (1, 23): 2,
 (1, 24): 2,
 (1, 25): 2,
 (1, 26): 2,
 (1, 27): 2,
 (1, 28): 2,
 (1, 29): 1,
 (1, 30): 2,
 (1, 31): 2,
 (2, 1): 1,
 (2, 2): 2,
 (2, 3): 3,
 (2, 4): 4,
 (2, 5): 5,
 (2, 6): 6,
 (2, 7): 7,
 (2, 8): 1,
 (2, 9): 2,
 (2, 10): 3,
 (2, 11): 4,
 (2, 12): 5,
 (2, 13): 6,
 (2, 14): 7,
 (2, 15): 1,
 (2, 16): 2,
 (2, 17): 3,
 (2, 18): 4,
 (2, 19): 5,
 (2, 20): 6,
 (2, 21): 7,
 (2, 22): 1,
 (2, 23): 2,
 (2, 24): 3,
 (2, 25): 4,
 (2, 26): 5,
 (2, 27): 6,
 (2, 28): 7,
 (3, 1): 1,
 (3, 2): 2,
 (3, 3): 3,
 (3, 4): 4,
 (3, 5): 5,
 (3, 6): 6,
 (3, 7): 7,
 (3, 8): 1,
 (3, 9): 2,
 (3, 10): 3,
 (3, 11): 4,
 (3, 12): 5,
 (3, 13): 6,
 (3, 14): 7,
 (3, 15): 1,
 (3, 16): 2,
 (3, 17): 3,
 (3, 18): 4,
 (3, 19): 5,
 (3, 20): 6,


### Concatenate DataFrame 20 Times and Upload to Snowflake

*Note: The following steps were not covered in the course. This is just one method to concatenate the DataFrame 20 times and upload it to Snowflake. Other approaches could involve using Pandas or the Snowsight UI, but I chose to complete it using only Snowpark and Python.*

Convert the dictionary to a CSV file

In [6]:
with open("pre.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["month", "day", "neighborhood"])
    for (month, day), neighborhood in pre.items():
        writer.writerow([month, day, neighborhood])

Create a Session with the necessary connection info

In [7]:
session = Session.builder.configs(params).create()

In [8]:
snowflake_env = session.sql("SELECT current_user(), current_version()").collect()
snowpark_version = VERSION

Create a virtual warehouse, database, and schema

In [9]:
session.sql("CREATE OR REPLACE WAREHOUSE FREEZING_POINT_WH WITH WAREHOUSE_SIZE='X-SMALL'").collect()
session.sql("USE WAREHOUSE FREEZING_POINT_WH").collect()
session.sql("CREATE OR REPLACE DATABASE FREEZING_POINT_DB").collect()
session.sql("CREATE OR REPLACE SCHEMA FREEZING_POINT_DB.FREEZING_POINT_SCHEMA").collect()

[Row(status='Schema FREEZING_POINT_SCHEMA successfully created.')]

In [10]:
print("Current Environment Details:")
print("---------------------------------")
print(f"User                        : {snowflake_env[0][0]}")
print(f"Role                        : {session.get_current_role()}")
print(f"Database                    : {session.get_current_database()}")
print(f"Schema                      : {session.get_current_schema()}")
print(f"Warehouse                   : {session.get_current_warehouse()}")
print(f"Snowflake version           : {snowflake_env[0][1]}")
print(f"Snowpark for Python version : {snowpark_version[0]}.{snowpark_version[1]}.{snowpark_version[2]}")
print("---------------------------------")

Current Environment Details:
---------------------------------
User                        : PREGISMOND
Role                        : "ACCOUNTADMIN"
Database                    : "FREEZING_POINT_DB"
Schema                      : "FREEZING_POINT_SCHEMA"
Warehouse                   : "FREEZING_POINT_WH"
Snowflake version           : 8.33.1
Snowpark for Python version : 1.18.0
---------------------------------


Next, we create a stage and put the CSV file into it.

In [11]:
session.sql("CREATE OR REPLACE STAGE FREEZING_POINT_STAGE").collect()
session.file.put(local_file_name="./pre.csv",
                 stage_location="@FREEZING_POINT_STAGE", 
                 auto_compress=False,
                 overwrite = True)

[PutResult(source='pre.csv', target='pre.csv', source_size=2928, target_size=2944, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

<img src="images/11_snowpark.png">

Create a table with defined schema

In [12]:
session.sql("""
CREATE OR REPLACE TABLE DF_PRE (
    Month NUMBER,
    Day NUMBER,
    Neighborhood NUMBER
);
""").collect()

[Row(status='Table DF_PRE successfully created.')]

Copy data from stage to table.

In [13]:
session.sql("""
COPY INTO DF_PRE
FROM @FREEZING_POINT_STAGE/pre.csv
FILE_FORMAT = (TYPE = 'CSV', FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER = 1);
""").collect()

[Row(file='freezing_point_stage/pre.csv', status='LOADED', rows_parsed=365, rows_loaded=365, error_limit=1, errors_seen=0, first_error=None, first_error_line=None, first_error_character=None, first_error_column_name=None)]

<img src="images/13_snowpark.png">

Create a DataFrame that points to `DF_PRE` table

In [14]:
df_pre = session.table("FREEZING_POINT_DB.FREEZING_POINT_SCHEMA.DF_PRE")

Concatenate the DataFrame with itself 20 times

* Using `union_all` to combine the DataFrames without removing duplicates

In [15]:
df_clean = df_pre
for _ in range(19):
    df_clean = df_clean.union_all(df_pre)

Save the combined DataFrame to Snowflake

In [16]:
df_clean.write.mode("overwrite").save_as_table("DF_CLEAN")

<img src="images/16_snowpark.png">

### Loading the data, processing the data, training the model, and evaluating

Create a DataFrame that points to `DF_CLEAN` table

In [17]:
snowpark_df = session.table("FREEZING_POINT_DB.FREEZING_POINT_SCHEMA.DF_CLEAN")

Show the first forty rows of the DataFrame

In [18]:
snowpark_df.show(n=40)

------------------------------------
|"MONTH"  |"DAY"  |"NEIGHBORHOOD"  |
------------------------------------
|1        |1      |1               |
|1        |2      |2               |
|1        |3      |2               |
|1        |4      |2               |
|1        |5      |2               |
|1        |6      |2               |
|1        |7      |2               |
|1        |8      |1               |
|1        |9      |2               |
|1        |10     |2               |
|1        |11     |2               |
|1        |12     |2               |
|1        |13     |2               |
|1        |14     |2               |
|1        |15     |1               |
|1        |16     |2               |
|1        |17     |2               |
|1        |18     |2               |
|1        |19     |2               |
|1        |20     |2               |
|1        |21     |2               |
|1        |22     |1               |
|1        |23     |2               |
|1        |24     |2               |
|

Count the rows in the DataFrame

* Check that we have 7,300 rows, which is 365 * 20

In [19]:
snowpark_df.count()

7300

We can generate descriptive statistics to view some basic statistical details like count, mean, standard deviation, etc.

Examine that the range of values are what we'd expect:

* 1 - 12 for months
* 1 - 31 for days
* 1 - 8 for neighborhoods

In [20]:
snowpark_df.describe().show()

---------------------------------------------------------------------------
|"SUMMARY"  |"MONTH"             |"DAY"              |"NEIGHBORHOOD"      |
---------------------------------------------------------------------------
|count      |7300.0              |7300.0             |7300.0              |
|mean       |6.526027            |15.720548          |4.019178            |
|stddev     |3.4480874408866145  |8.796849549696756  |2.2738080393911884  |
|min        |1.0                 |1.0                |1.0                 |
|max        |12.0                |31.0               |8.0                 |
---------------------------------------------------------------------------



Group by Neighborhood, and show the counts

In [21]:
snowpark_df.group_by("Neighborhood").count().show()

----------------------------
|"NEIGHBORHOOD"  |"COUNT"  |
----------------------------
|1               |1080     |
|2               |1500     |
|3               |900      |
|4               |800      |
|5               |800      |
|6               |800      |
|7               |800      |
|8               |620      |
----------------------------



As we can see above, neighborhood 2 is the most popular as it gets visited the most each January. Neighborhood 8 gets visited each December, but is never visited any other time during the year, so it has the lowest count.

With our data prepared, we can now use Snowpark ML to predict which neighborhood the food truck will visit on any given day in the future.

The `XGBClassifier` requires labels to be in the range of 0-7, rather than 1-8, which is how our neighborhoods are currently numbered. To scale the target "Neighborhood", we can use `LabelEncoder`, which takes an input column and produces an output column, automatically subtracting one from each value.

Next, we apply the `fit` method on this LabelEncoder. Finally, we use the `transform` method on the fitted Snowpark DataFrame.

In [22]:
# Use scikit-learn's LabelEncoder -- a more general solution -- through Snowpark ML
le = preprocessing.LabelEncoder(input_cols=['NEIGHBORHOOD'], output_cols= ['NEIGHBORHOOD2'], drop_input_cols=True)

# Apply the LabelEncoder
fitted = le.fit(snowpark_df.select("NEIGHBORHOOD"))

snowpark_df_prepared = fitted.transform(snowpark_df)

Show the DataFrame

In [23]:
snowpark_df_prepared.show()

-------------------------------------
|"NEIGHBORHOOD2"  |"MONTH"  |"DAY"  |
-------------------------------------
|0.0              |1        |1      |
|1.0              |1        |2      |
|1.0              |1        |3      |
|1.0              |1        |4      |
|1.0              |1        |5      |
|1.0              |1        |6      |
|1.0              |1        |7      |
|0.0              |1        |8      |
|1.0              |1        |9      |
|1.0              |1        |10     |
-------------------------------------



Split the data into a training set and a test set

Here we're using `randomSplit` from Snowpark ML, even though the underlying functionality is derived from scikit-learn. We'll set aside 10% of our 20 years of truck location data for testing.

In [24]:
train_snowpark_df, test_snowpark_df = snowpark_df_prepared.randomSplit([0.9, 0.1])

Save training data as `DF_CLEAN_TRAIN` and test data as `DF_CLEAN_TEST` to our Snowflake instance.

In [25]:
# Save training data
train_snowpark_df.write.mode("overwrite").save_as_table("DF_CLEAN_TRAIN")

# Save test data
test_snowpark_df.write.mode("overwrite").save_as_table("DF_CLEAN_TEST")

<img src="images/25_snowpark.png">

<img src="images/25.1_snowpark.png">

Create and train the `XGBClassifier` model

Here we provide our days and months data along with the neighborhood target variable to the `XGBClassifier`. The classifier will use this information to predict the neighborhood where the truck is located.

The feature columns are the columns the `XGBClassifier` will use to learn and predict the label column.

Next, we call the `fit` method on the model, passing our DataFrame as the argument. This `fit` method initiates the training process.

In [26]:
FEATURE_COLS = ["MONTH", "DAY"]
LABEL_COLS = ["NEIGHBORHOOD2"]

# Train an XGBoost model on snowflake.
xgboost_model = XGBClassifier(
    input_cols=FEATURE_COLS,
    label_cols=LABEL_COLS
)

xgboost_model.fit(train_snowpark_df)

Check the accuracy using scikit-learn's `score` functionality through Snowpark ML

The truck driver schedule was deliberately made very regular to test how well XGBoost can detect strange yet consistent patterns in the data.

In [27]:
accuracy = xgboost_model.score(test_snowpark_df)

print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 100.00%


The XGBoost classifier we trained, when applied to the test set, made 730 predictions and got 730 of those correct.

### Register Model to Snowflake Model Registry

*Note: The following step was not included in the course. We can load our model into the Snowflake Model Registry to ensure easy access later and to keep track of all our models.*

Load `xgboost_model` into the Snowflake Model Registry with key metrics.

In [28]:
# Register the model to the Snowflake model registry
registry = Registry(session)

# Define key metrics
key_metrics = {"accuracy": accuracy}

# Log model to registry
model_ref = registry.log_model(
    xgboost_model,
    model_name="FREEZING_POINT_MODEL",
    version_name="v1",
    metrics=key_metrics,
    options={"relax_version": True}
)

<img src="images/28.0_snowpark.png">

<img src="images/28.1_snowpark.png">

### Stop Snowpark Session

In [29]:
session.close()

## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2024-08-19|0.2|Pravin Regismond|Modified to include:<br>- calling credential.py containing a dictionary of credentials<br>- concatenate DataFrame 20 times and upload to Snowflake<br>- register model to Snowflake Model Registry|
|2023-04-27|0.1|Peter Olson|Initial Version|
